In [32]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
from alibi.explainers import IntegratedGradients, CounterfactualProto
from sklearn.inspection import permutation_importance
import json


In [33]:
# === Load and Merge Data ===
# Load datasets
df_main = pd.read_csv('final.csv')
df_main['statecounty'] = df_main['statecounty'].astype(str).apply(lambda x: x.split()[0])


In [34]:
df_target = pd.read_csv('cvd_mortality_2018_2020.csv', index_col=0)
df_target['statecounty'] = df_target['statecounty'].apply(lambda x: x.split()[0])


In [35]:
# Remove duplicates in target data
df_target = df_target.drop_duplicates(subset='statecounty', keep='first')


In [36]:
# Impute missing values in target columns
allRaceCVD = df_target.columns[2]
APICVD = df_target.columns[1]
df_target[allRaceCVD] = df_target[allRaceCVD].fillna(df_target[APICVD])


In [37]:
# Merge datasets
df = pd.merge(df_main, df_target, on='statecounty', how='inner')
print(f"Number of rows after merging: {len(df)}")


Number of rows after merging: 3206


In [38]:
# Identify islands (specific condition, optional)
islands = df['statecounty'].apply(lambda x: x.startswith('15'))
islandnames = df[islands]['statecounty']


In [39]:
# Remove statecounty column after processing
df = df.drop(columns=['statecounty'])


In [40]:
# === Select Relevant Columns ===
# Select columns containing 'Per' or 'per' in their names
columns_to_keep = [
    col for col in df.columns if ('Per' in col or 'per' in col) and 'Upper' not in col and 'Housing' not in col
]
columns_to_keep.append('NHPI_Single_Race_Civilian_Population_With_Health_Insurance')
target = 'allrace_cvd_mortality_per_100k_2018_2020'


In [41]:
# Filter dataset
df = df[columns_to_keep]


In [42]:
# === Data Cleaning ===
# Replace infinite values and outliers
large_value_threshold = 1e308
df = df.replace([np.inf, -np.inf], np.nan)
df = df.applymap(lambda x: np.nan if isinstance(x, float) and abs(x) > large_value_threshold else x)


/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_9704/3084115497.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: np.nan if isinstance(x, float) and abs(x) > large_value_threshold else x)


In [43]:
# Drop rows where all columns are NaN
df.dropna(how='all', inplace=True)


In [44]:
# Separate target variable
y = df[target]
X = df.drop(columns=[target])


In [45]:
# Identify numeric and categorical columns
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()


In [46]:
print(f"Numeric columns: {len(numeric_cols)}, Categorical columns: {len(categorical_cols)}")


Numeric columns: 495, Categorical columns: 1


In [47]:
# === Preprocessing Pipelines ===
# Numeric preprocessing
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])


In [48]:
# Categorical preprocessing
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])


In [49]:
# Combine preprocessors
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
])


In [50]:
# === Apply Preprocessing ===
# Transform features
X_processed = preprocessor.fit_transform(X)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['NHPI_Percent_Population_Under_Poverty_ACS2017_21'
 'NHPI_Percent_Population_Under_18_Under_Poverty_ACS2017_21'
 'AIAN_Average_Performance_For_3rd_Graders_Reading_Tests_2016_CHR'
 'AIAN_Average_Performance_For_3rd_Graders_Math_Tests_2016_CHR']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [51]:
# Validate processed data
assert not np.any(np.isnan(X_processed)), "Processed data contains NaN values."
assert not np.any(np.isinf(X_processed)), "Processed data contains infinity values."


In [52]:
# Impute missing values in the target variable
y = y.replace([np.inf, -np.inf], np.nan)
y = pd.to_numeric(y, errors='coerce')


In [53]:
imputer_y = SimpleImputer(strategy='mean')
y_imputed = pd.Series(imputer_y.fit_transform(y.values.reshape(-1, 1)).flatten(), name=target)


In [54]:
# === Split Data ===
X_train, X_val, y_train, y_val = train_test_split(X_processed, y_imputed, test_size=0.2, random_state=42)


In [55]:
# === Advanced model training and tuning ===


In [56]:
# Import necessary libraries
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
import json


In [57]:
# === Model Training and Evaluation ===


In [58]:
# Define and initialize models
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
catboost_model = CatBoostRegressor(verbose=0, random_state=42)


In [59]:
# Stacking ensemble for improved performance
stacked_model = StackingRegressor(
    estimators=[
        ('xgb', xgb_model),
        ('catboost', catboost_model)
    ],
    final_estimator=XGBRegressor(random_state=42)
)


In [60]:
# Train the models
print("Training XGBoost model...")
xgb_model.fit(X_train, y_train)


Training XGBoost model...


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [61]:
print("Training CatBoost model...")
catboost_model.fit(X_train, y_train)
print("Training Stacked model...")
stacked_model.fit(X_train, y_train)


Training CatBoost model...
Training Stacked model...


StackingRegressor(estimators=[('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_ra...
                                               grow_policy=None,
                                               importance_type=None,
                                               interaction_constraints=None,
                                               learning_rate=None, max_bin=None,
                                               max_cat_threshold=None,
                                               max_cat_to_onehot=None,
                                               max_delta_step=None,
                                               max_depth=None, max_leaves=None,
                                               min_child_weight=None,
                                               missing=nan,
                                               monotone_constraints=None,
                                               multi_strategy=None,
                                               n_estimators=None, n_jobs=None,
                                               num_parallel_tree=None,
                                               random_state=42, ...))

In [62]:
# === Model Evaluation ===
models = {
    'XGBoost': xgb_model,
    'CatBoost': catboost_model,
    'Stacked': stacked_model
}


In [63]:
# Evaluate all models on the validation set
results = {}
for name, model in models.items():
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    explained_var = explained_variance_score(y_val, y_pred)
    results[name] = {'MSE': mse, 'R2': r2, 'Explained Variance': explained_var}
    print(f"{name} Results: MSE={mse:.4f}, R2={r2:.4f}, Explained Variance={explained_var:.4f}")
        # Save predictions for external analysis
    np.savetxt(f'{name}_predictions.csv', np.c_[y_val, y_pred], delimiter=',', header='True,Predicted', comments='')


XGBoost Results: MSE=553.8536, R2=0.9292, Explained Variance=0.9292
CatBoost Results: MSE=435.6144, R2=0.9443, Explained Variance=0.9444
Stacked Results: MSE=546.8344, R2=0.9301, Explained Variance=0.9302


In [64]:
# Save evaluation metrics to a JSON file
with open('model_results.json', 'w') as f:
    json.dump(results, f)


In [65]:
# === Explainability with Feature Importances ===
from sklearn.inspection import permutation_importance


In [66]:
print("Calculating feature importances for Stacked model...")
perm_importances = permutation_importance(stacked_model, X_val, y_val, n_repeats=10, random_state=42)
feature_importances = {
    feature: importance for feature, importance in zip(preprocessor.get_feature_names_out(), perm_importances.importances_mean)
}


Calculating feature importances for Stacked model...


In [67]:
# Save feature importances for analysis
with open('feature_importances.json', 'w') as f:
    json.dump(feature_importances, f)


In [68]:
# === Summary ===
print("\nModel Evaluation Summary:")
for name, metrics in results.items():
    print(f"{name} - MSE: {metrics['MSE']:.4f}, R2: {metrics['R2']:.4f}, Explained Variance: {metrics['Explained Variance']:.4f}")



Model Evaluation Summary:
XGBoost - MSE: 553.8536, R2: 0.9292, Explained Variance: 0.9292
CatBoost - MSE: 435.6144, R2: 0.9443, Explained Variance: 0.9444
Stacked - MSE: 546.8344, R2: 0.9301, Explained Variance: 0.9302
